# Packages imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV

# Data Import
2022 legislatives, explaining far right vote

In [ ]:
path = "../Data/data_merged_20250922.parquet"

sel = [("type","==",0),("annee","==",2022)]

col = ["annee", "type", "codecommune", "inscrits", "lat", "long",
           "pvoteppar", "pvotepreviousppar", "pvotepreviouspreviousppar", "pvotepreviouspreviouspvoteC", "pvotepreviouspreviouspvoteCD", "pvotepreviouspreviouspvoteCG", "pvotepreviouspreviouspvoteD", "pvotepreviouspreviouspvoteG", "pvotepreviouspvoteC", "pvotepreviouspvoteCD", "pvotepreviouspvoteCG", "pvotepreviouspvoteD", "pvotepreviouspvoteG", "pvotepvoteD",
           "popcommunes/pop", "popcommunesvbbm/vbbm",
           "agesexcommunes/prop014","agesexcommunes/prop60p","agesexcommunes/perage",
           "diplomescommunes/pbac", "diplomescommunes/psup", "diplomescommunes/nodip",
           "cspcommunes/pouvr","cspcommunes/pcadr","cspcommunes/pchom",
           "revcommunes/revratio",
           "rsacommunes/perrsa",
           "capitalimmobiliercommunes/prixm2ratio",
           "naticommunes/pimmigre"]

# data = pd.read_parquet(path,engine='pyarrow',columns=col,filters=sel)
data = pd.read_parquet(path)

# path = "../Data/data_no_na.pkl"
# dataNoNa = pd.read_pickle(path)

# X distribution estimation

## Test with simulated data

In [3]:
n = 1000
cov = np.array([[ 1, -0.5, 0 ],[-0.5, 1, 0.5 ],[ 0, 0.5, 1 ]])
s = [0,0,0]
G = np.block([[cov,cov-np.diag(s)],[cov-np.diag(s),cov]])

mu = np.array([-1, 1, 3])
X = np.random.multivariate_normal(mu,cov,n)

Y = X[:,0] - X[:,1] + np.random.normal(0,0.1,n) #+ 2*X[:,2] 

mu_hat = X - X@np.linalg.inv(cov)@np.diag(s)
cov_hat = 2*np.diag(s) - np.diag(s)@np.linalg.inv(cov)@np.diag(s)
X_hat = np.empty_like(mu_hat)
for i in range(n):
    X_hat[i] = np.random.multivariate_normal(mu_hat[i], cov_hat)

X_tot = np.column_stack((X,X_hat))

reg = LassoCV().fit(X_tot,Y)
coeff = reg.coef_

## Using Knockpy

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler

data = data.dropna(axis=0)
y = data["pvotepvoteD"].to_numpy(dtype=np.float64)
new_data = data.drop(columns=['pvotepvoteD','annee','type'])
X = new_data.to_numpy(dtype = np.float64)

headers = new_data.columns.values.tolist()
rvm = []
for i in range(len(headers)):
    if np.std(X[:,i]) < 1e-1:
        rvm.append(i)
new_data = new_data.drop(columns=[headers[i] for i in rvm])

X = new_data.to_numpy(dtype = np.float64)
X = StandardScaler().fit_transform(X)
# X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1)
X = np.ascontiguousarray(X, dtype=np.float64)

print([headers[i] for i in rvm])
    

['pvoteppar', 'pvotepreviousppar', 'pvotepreviouspreviousppar', 'pvotepreviouspreviouspvoteC', 'pvotepreviouspreviouspvoteCD', 'pvotepreviouspreviouspvoteCG', 'pvotepreviouspreviouspvoteD', 'pvotepreviouspreviouspvoteG', 'pvotepreviouspvoteC', 'pvotepreviouspvoteCD', 'pvotepreviouspvoteCG', 'pvotepreviouspvoteD', 'pvotepreviouspvoteG', 'agesexcommunes/prop014', 'naticommunes/pimmigre']


In [ ]:
import knockpy as kpy
from knockpy.knockoff_filter import KnockoffFilter

kfilter = KnockoffFilter(ksampler="fx",fstat="mlr",knockoff_kwargs={'method':'equicorrelated'})

rejections = kfilter.forward(X=X_test,y=y_test,fdr=0.1)

print(rejections)

## Using Knockofforigins

In [7]:
from knockofforigins.knockoff_construct import generate_knockoff_features, choose_s_vector

s = choose_s_vector(np.cov(X.T))
Xk = generate_knockoff_features(X,s)

ValueError: Matrix must be positive-definite for Cholesky decomposition.

In [ ]:
from knockofforigins.lasso import compute_feature_importance

# Compute feature importance statistic W
alpha = 0.1  # Regularization parameter for Lasso
W = compute_feature_importance(X, Xk, y, alpha)